# Setup
Install necessary packages

In [17]:
!pip install wandb torch ultralytics scikit-image

In [15]:
!pip uninstall torch torchvision
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu117

^C
Looking in indexes: https://download.pytorch.org/whl/cu117


In [3]:
import os
from pathlib import Path
import shutil

import torch
import pandas as pd
import numpy as np
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, datasets

from ultralytics import YOLO
from ultralytics.yolo.engine.trainer import BaseTrainer

import wandb

# Data Preprocessing
Set up data for training and validation

In [21]:
cwd = Path.cwd()
datasets_path = cwd / "datasets"
prannays_edibles_path = datasets_path / "prannays_edibles"
print(prannays_edibles_path)

c:\Users\Francis Ralph\Desktop\Misc\BuildingBloCS\BBCS-June-Intermediate\datasets\prannays_edibles


In [22]:
class_name_map = {
    '0': 'bread',
    '1': 'dairy',
    '2': 'dessert',
    '3': 'egg',
    '4': 'fried',
    '5': 'meat',
    '6': 'pasta',
    '7': 'rice',
    '8': 'seafood',
    '9': 'soup',
    '10': 'vegetables',
}

In [102]:
prannays_edibles_dataset = datasets.ImageFolder(root=prannays_edibles_path)

train_split_percentage = 0.6 # TODO: try 80-20 split, then 60-40, then try equalizing the amount of classes
test_split_percentage = 1 - train_split_percentage

train_dataset, test_dataset = random_split(prannays_edibles_dataset, [train_split_percentage, test_split_percentage])

In [103]:
def get_class_name(class_index):
    idx_to_class = {v: k for k, v in prannays_edibles_dataset.class_to_idx.items()}
    return class_name_map[idx_to_class[class_index]]


In [104]:
split_dataset_path = cwd / "datasets" / "prannays_edibles_split"
if split_dataset_path.exists() and split_dataset_path.is_dir():
    shutil.rmtree(split_dataset_path) # reset split

split_dataset_path.mkdir(exist_ok=True)

train_dataset_path = split_dataset_path / "train"
train_dataset_path.mkdir(exist_ok=True)

test_dataset_path = split_dataset_path / "test"
test_dataset_path.mkdir(exist_ok=True)

def create_dataset_folder(dataset, dataset_path):
    for i, (image, image_class_idx) in enumerate(dataset):
        image_class_name = get_class_name(image_class_idx)
        class_path = dataset_path / str(image_class_name)
        if not class_path.exists():
            class_path.mkdir(exist_ok=True)
        image.save(class_path / f"{image_class_name}_{i}.jpg")

create_dataset_folder(train_dataset, train_dataset_path)
create_dataset_folder(test_dataset, test_dataset_path)

# Model Training

In [105]:
model_path = Path('yolov8n-cls.pt')
if model_path.exists():
    model_path.unlink()
model = YOLO('yolov8n-cls.pt') # load pretrained model

100%|██████████| 5.28M/5.28M [00:03<00:00, 1.71MB/s]


In [117]:
# login to wandb to monitor training metrics
os.environ["WANDB_API_KEY"] = input()
wandb.init(project='BuildingBloCS Prannays Edibles Classifier', settings=wandb.Settings(start_method="spawn"), mode='online')

lr/pg0,█▅▂▂▂▂▂▁▁▁▁▁▁▁▁
lr/pg1,▃▆█▇▇▇▆▅▅▄▄▃▂▂▁
lr/pg2,▃▆█▇▇▇▆▅▅▄▄▃▂▂▁
metrics/accuracy_top1,▁▂▃▅▅▇▇▇█▇█████
metrics/accuracy_top5,▁▂▃▅▅▇▇▇█▇██▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,██▇▇▆▆▆▅▅▄▄▃▂▂▁
val/loss,█▇▇▅▅▄▃▂▂▂▂▁▁▁▁
lr/pg0,0.00142


In [107]:
epochs = 15
batch = 4 # TODO: change to 1

model.train(data=str(split_dataset_path), batch=batch, epochs=epochs, save_period=5)

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=c:\Users\Francis Ralph\Desktop\Misc\BuildingBloCS\BBCS-June-Intermediate\datasets\prannays_edibles_split, epochs=15, patience=50, batch=4, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchs

WARNING  ConfusionMatrix plot failure: No module named 'ipympl'


                   all      0.527      0.907

15 epochs completed in 1.841 hours.
Optimizer stripped from runs\classify\train12\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train12\weights\best.pt, 3.0MB
Results saved to runs\classify\train12
wandb: Network error (ConnectionError), entering retry loop.


# Model Validation

In [151]:
metrics = model.val(data=str(split_dataset_path)) # TODO: gather external data for validation dataset

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu CPU
WARNING  Dataset 'split=val' not found, using 'split=test' instead.
               classes   top1_acc   top5_acc: 100%|██████████| 740/740 [01:04<00:00, 11.46it/s]
                   all       0.51      0.925
Speed: 0.0ms preprocess, 14.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val11


In [150]:
print("metrics", metrics)

metrics ultralytics.yolo.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x000001B248C8BB50>
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
speed: {'preprocess': 0.0, 'inference': 266.00122451782227, 'loss': 0.0, 'postprocess': 0.0}
top1: 0.0
top5: 1.0


# Model Prediction

In [145]:
# results = model('datasets/prannays_edibles_split/test/soup/soup_989.jpg')
results = model('images_to_predict/prannays_edibles/1200-Perfect-Ribeye-Steak-SpendWithPennies.jpg')


image 1/1 c:\Users\Francis Ralph\Desktop\Misc\BuildingBloCS\BBCS-June-Intermediate\images_to_predict\prannays_edibles\1200-Perfect-Ribeye-Steak-SpendWithPennies.jpg: 224x224 dessert 0.66, seafood 0.16, meat 0.11, bread 0.05, vegetables 0.01, 34.0ms
Speed: 9.0ms preprocess, 34.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [146]:
for result in results:
    for i, contender in enumerate(result.probs.top5):
        print(i, result.names[contender], f"({result.probs.top5conf[i] * 100:.2f}% confidence)")

0 dessert (65.83% confidence)
1 seafood (15.90% confidence)
2 meat (11.47% confidence)
3 bread (4.59% confidence)
4 vegetables (1.30% confidence)
